### Toward Handling a Common Data Pattern with R data.table. 

I just updated this Jupyter notebook on [Medicare provider utilization and payment](https://www.cms.gov/research-statistics-data-systems/medicare-provider-utilization-and-payment-data/medicare-provider-utilization-and-payment-data-physician-and-other-supplier/physician-and-other-supplier-data-cy-2018) I've maintained for a number of years. In my data science travels, I see a pattern like the one presented here a lot: Text data files are added annually (quarterly, etc.) with a lag. (Coverage in this instance is currently from 2012-2018 inclusive.) Data attribute names don't change from year to year, though variables can either be added or removed. The data management challenge is then to load and stack each year's data adding a file/time identifier, the final attribute list representing the union of each year's, with clear demarcation of missing data.

I've been using R for almost 20 years and its [splendid data.table package](https://cran.r-project.org/web/packages/data.table/vignettes/datatable-intro.html) for 10. For my money, data.table is the most mature open-source data handling package extant, it's loading/grouping/summarizing capabilities nonpareil. Though I like python/pandas, I think data.table is even more powerful and performant. Actually, I prefer collaborating the two using a common data file protocol like parquet along with interop packages such as rpy2 and reticulate. I'll oftentimes load data using R/data.table, subsequently writing  parquet files for sharing with other platforms.

A design quandary with R data loads is how to handle character fields. One can store string attributes as either character or factor in R dataframes/data.tables. Factors are integer pointers to the value strings and can often save storage. The longer the string values and the lower the cardinality, the larger the storage savings. That said, I've generally found that factors are preferred to character even if attribute cardinality exceeds 50% of the number of table rows. Add to that read/write performance advantages and the case for factors as default can be compelling, as we'll see below.

What follows is basic R data.table code to handle the pattern articulated above. data.table's fread text data loader is comprehensive, and its rbindlist functionality seamlessly manages the stacking of the disparate input files. Once the data are loaded, R's dynamic string execution along with data.table's efficient grouping capabilities make the critical exploratory frequencies work a snap. And finally, writing the data.tables to parquet files promotes efficient interoperability with other analytics platforms. At the conclusion of the read, my hope is that readers take away an appreciation of the power of data.table and its ease of use as a basis for split/apply/combine work, not only in R but in other analytics software as well.  

The supporting platform is a Wintel 10 notebook with 128 GB RAM, along with software JupyterLab 3.0.7 and R 4.0.2. The R data.table, tidyverse, purrr, knitr, and arrow  packages are featured, in addition to functions meta and freqsdt from my personal stash, detailed below.

R library paths.

In [25]:
.libPaths()

[1] "C:/R/4.0.2/library"                 "C:/Program Files/R/R-4.0.2/library"

Set options and load relevant R libraries. Source a personal R function file.

In [26]:
options(warn=-1) 
options(scipen = 20)
options(datatable.print.topn=100)
options(datatable.showProgress=FALSE)
options(stringsAsFactors=TRUE)

usualsuspects <- c(
     'data.table', 'pryr', 'plyr','readxl', 'bit', 'dtplyr',   
    'rvest', 'magrittr','lubridate','rlist', 'dplyr', 'purrr',
    'tidyquant', 'tidyverse', 'tictoc',
    'fst','feather','arrow',
    'knitr', 'kableExtra',  
    'ggplot2','RColorBrewer',
    'patchwork','hrbrthemes'
) 

suppressMessages(invisible(lapply(usualsuspects, library, character.only = TRUE)))

funcsdir <- "c:/steve/r/functions"
funcsfile <- "rfunctions.r"

setwd(funcsdir)
source(funcsfile)


blanks(1)
lsf.str()

blanks(2)

allfreqs : function (dtn, catlim = 100)  
blanks : function (howmany)  
colsize : function (dt)  
dtmeta : function (df)  
freqsdt : function (DTstr, xstr, percent = TRUE)  
load_dta : function (fname, howmany = Inf, saf = TRUE)  
meta : function (df, data = FALSE, dict = TRUE)  
mksumfreq : function (freqalldt)  
mksumfreq2 : function (dt)  
mykab : function (dt)  
obj_sz : function (obj)  
prhead : function (df, howmany = 6)  
prheadtail : function (df, howmany = 6)  

Migrate to the working directory. Identify the raw data files.

In [27]:
mdir <- "d:/bigdata/raw/medicare"
setwd(mdir)

fnmes <- Sys.glob("M*.txt")

blanks(2)

How big's the raw data? 

In [28]:
GIG <- 1024**3

howbig <- round(sum(map_dbl(fnmes,function(n) file.info(n)$size))/GIG,2)
paste0(howbig,' G')

blanks(2)

[1] "14.19 G"

Define the data.table meta data function sourced earlier.

In [29]:
meta <- function(df,data=FALSE,dict=TRUE)
{
    HOWMANY <- 12
    slug=data.table(name=deparse(substitute(df)),class=list(class(df)),rows=dim(df)[1],columns=dim(df)[2],size=obj_sz(df))
    print(mykab(slug))
    blanks(1)
    if (dict) print(str(df))

    if (data)
        if (slug$rows<HOWMANY)
            print(mykab(df)) 
        else
        {
            print(mykab(head(df))) 
            print(mykab(tail(df))) 
        }
    blanks(1)
}

blanks(2)

Function invoked to load each raw data file by rbindlist.

In [30]:
load_dta <- function(fname,howmany=Inf,saf=TRUE)
{
    header <- tolower(fread(fname,nrows=1,header=FALSE))
    slug <- fread(fname,header=FALSE,skip=2,nrows=howmany,stringsAsFactors=saf)
    names(slug) <- header
    start <- str_locate(fname,"_CY")[2]+1
    slug$year <- as.integer(substring(fname,start,start+3))
    return(slug)
}

blanks(2)

Load all files into the medicaref data.table with strings as factors.

In [31]:
tic()

medicaref <- rbindlist(lapply(fnmes, load_dta,saf=TRUE),
                      use.names=TRUE, fill=TRUE)
meta(medicaref)

toc()

blanks(2)



|name     |class                 |rows    |columns|size   |
|:--------|:---------------------|:-------|:------|:------|
|medicaref|data.table, data.frame|66779550|30     |9.55 GB|

Classes 'data.table' and 'data.frame':	66779550 obs. of  30 variables:
 $ npi                             : int  1003000126 1003000126 1003000126 1003000126 1003000126 1003000126 1003000126 1003000134 1003000134 1003000134 ...
 $ nppes_provider_last_org_name    : Factor w/ 318343 levels "10 33 AMBULANCE SERVICE LIMITED",..: 60738 60738 60738 60738 60738 60738 60738 37340 37340 37340 ...
 $ nppes_provider_first_name       : Factor w/ 81568 levels "","(BARBARA)",..: 3265 3265 3265 3265 3265 3265 3265 48840 48840 48840 ...
 $ nppes_provider_mi               : Factor w/ 34 levels "","'","(","-",..: 1 1 1 1 1 1 1 19 19 19 ...
 $ nppes_credentials               : Factor w/ 22866 levels "","(D.C.) CHIROPRACTIC",..: 5088 5088 5088 5088 5088 5088 5088 5088 5088 5088 ...
 $ nppes_provider_gender           : Factor w

Load the same files into the medicares data.table with strings as strings. Note the almost double load time and > 40% additional storage cost for the string data version.

In [32]:
tic()

medicares <- rbindlist(lapply(fnmes, load_dta,saf=FALSE),
                      use.names=TRUE, fill=TRUE)
meta(medicares)

toc()

blanks(2)



|name     |class                 |rows    |columns|size    |
|:--------|:---------------------|:-------|:------|:-------|
|medicares|data.table, data.frame|66779550|30     |14.02 GB|

Classes 'data.table' and 'data.frame':	66779550 obs. of  30 variables:
 $ npi                             : int  1003000126 1003000126 1003000126 1003000126 1003000126 1003000126 1003000126 1003000134 1003000134 1003000134 ...
 $ nppes_provider_last_org_name    : chr  "ENKESHAFI" "ENKESHAFI" "ENKESHAFI" "ENKESHAFI" ...
 $ nppes_provider_first_name       : chr  "ARDALAN" "ARDALAN" "ARDALAN" "ARDALAN" ...
 $ nppes_provider_mi               : chr  "" "" "" "" ...
 $ nppes_credentials               : chr  "M.D." "M.D." "M.D." "M.D." ...
 $ nppes_provider_gender           : chr  "M" "M" "M" "M" ...
 $ nppes_entity_code               : chr  "I" "I" "I" "I" ...
 $ nppes_provider_street1          : chr  "900 SETON DR" "900 SETON DR" "900 SETON DR" "900 SETON DR" ...
 $ nppes_provider_street2          : chr  "" 

Now write parquet files for each of the Medicare data.tables. parquet is lingua franca for data sharing among analytics management packages. Note the stark difference in elapsed times between the string and factor data.table versions. Note also how much smaller the parquet file sizes are than the original raw data -- in these instances about an 80% reduction.

In [33]:
tic()

fn <- "medicarers.parquet"
write_parquet(medicares,fn)

paste0(round(file.info(fn)$size/GIG,2),' G')
blanks(1)

toc()

blanks(2)

[1] "2.83 G"


302.5 sec elapsed

 


In [34]:
tic()

fn <- "medicarerf.parquet"
write_parquet(medicaref,fn)

paste0(round(file.info(fn)$size/GIG,2),' G')
blanks(1)

toc()

blanks(2)

[1] "2.64 G"


35.77 sec elapsed

 


Frequencies, counts of variable intersections, are the most elemental of grouping operations. freqsdt is a general-purpose frequency function built on dynamic build/execution of data.table grouping counts that is simple but powerful.

In [35]:
freqsdt <- function(DTstr, xstr, percent=TRUE)
{    
    if (percent)
        return(eval(parse(text=sprintf('%s[,.(frequency=.N),.(%s)]', DTstr, xstr)))[
            order(-frequency)][,percent:=100*frequency/sum(frequency)]) 
    else
        return(eval(parse(text=sprintf('%s[,.(frequency=.N),.(%s)]', DTstr, xstr)))[
            order(-frequency)]) 
}   

blanks(2)

nppes_credentials frequencies with freqsdt. very fast.

In [36]:
tic()

dt = "medicaref"
cols = "nppes_credentials"

fc <- freqsdt(dt,cols)
meta(fc,data=TRUE)

toc()

blanks(2)



|name|class                 |rows |columns|size   |
|:---|:---------------------|:----|:------|:------|
|fc  |data.table, data.frame|22867|3      |1.97 MB|

Classes 'data.table' and 'data.frame':	22867 obs. of  3 variables:
 $ nppes_credentials: Factor w/ 22866 levels "","(D.C.) CHIROPRACTIC",..: 6579 5088 1 2346 3096 10203 3246 9441 1690 4982 ...
 $ frequency        : int  23200195 22378848 4489350 2229095 1704125 1034678 881343 668417 657737 570015 ...
 $ percent          : num  34.74 33.51 6.72 3.34 2.55 ...
 - attr(*, ".internal.selfref")=<externalptr> 
NULL


|nppes_credentials|frequency|percent  |
|:----------------|:--------|:--------|
|MD               |23200195 |34.741467|
|M.D.             |22378848 |33.511529|
|                 |4489350  |6.722642 |
|D.O.             |2229095  |3.337990 |
|DO               |1704125  |2.551867 |
|PA-C             |1034678  |1.549393 |


|nppes_credentials|frequency|percent |
|:----------------|:--------|:-------|
|PHD, GNP-BC      |1       

nppes_provider_zip, a 2nd single attribute illustration. There are almost 370K different "zips" represented in the data. That 370K is of course the cardinality of nppes_provider_zip.

In [37]:
tic()

dt = "medicaref"
cols = "nppes_provider_zip"

fc <- freqsdt(dt,cols)
meta(fc,data=TRUE)

toc()

blanks(2)



|name|class                 |rows  |columns|size    |
|:---|:---------------------|:-----|:------|:-------|
|fc  |data.table, data.frame|369899|3      |30.88 MB|

Classes 'data.table' and 'data.frame':	369899 obs. of  3 variables:
 $ nppes_provider_zip: Factor w/ 369898 levels "0000","00000",..: 157355 126839 90279 92059 64438 149446 111995 198131 151880 5466 ...
 $ frequency         : int  136500 98294 45660 44221 44128 42966 42593 42313 41937 40592 ...
 $ percent           : num  0.2044 0.1472 0.0684 0.0662 0.0661 ...
 - attr(*, ".internal.selfref")=<externalptr> 
NULL


|nppes_provider_zip|frequency|percent |
|:-----------------|:--------|:-------|
|559050001         |136500   |0.204404|
|441950001         |98294    |0.147192|
|322241865         |45660    |0.068374|
|326103003         |44221    |0.066219|
|229080001         |44128    |0.066080|
|522421009         |42966    |0.064340|


|nppes_provider_zip|frequency|percent |
|:-----------------|:--------|:-------|
|45453          

Multiple attribute frequencies of nppes_credentials with nppes_provider_zip. Almost 1M intersections, but still very fast.

In [38]:
tic()

dt = "medicaref"
cols = "nppes_credentials,nppes_provider_zip"

fc <- freqsdt(dt,cols)

meta(fc,data=TRUE)

toc()

blanks(2)



|name|class                 |rows  |columns|size    |
|:---|:---------------------|:-----|:------|:-------|
|fc  |data.table, data.frame|959410|4      |45.15 MB|

Classes 'data.table' and 'data.frame':	959410 obs. of  4 variables:
 $ nppes_credentials : Factor w/ 22866 levels "","(D.C.) CHIROPRACTIC",..: 5088 6579 6579 6579 6579 6579 5088 6579 6579 5088 ...
 $ nppes_provider_zip: Factor w/ 369898 levels "0000","00000",..: 157355 126839 172708 53962 92059 111995 61626 157355 61567 64438 ...
 $ frequency         : int  83051 51688 32286 31025 30886 27879 27836 26777 26222 26057 ...
 $ percent           : num  0.1244 0.0774 0.0483 0.0465 0.0463 ...
 - attr(*, ".internal.selfref")=<externalptr> 
NULL


|nppes_credentials|nppes_provider_zip|frequency|percent |
|:----------------|:-----------------|:--------|:-------|
|M.D.             |559050001         |83051    |0.124366|
|MD               |441950001         |51688    |0.077401|
|MD               |631101032         |32286    |0.048347|


frequencies of nppes_provider_zip with a data.table subset. 

In [39]:
tic()

dt = "medicaref[year==2012]"
cols = "nppes_provider_zip"

fc <- freqsdt(dt,cols)

meta(fc,data=TRUE)

toc()

blanks(2)



|name|class                 |rows  |columns|size    |
|:---|:---------------------|:-----|:------|:-------|
|fc  |data.table, data.frame|258125|3      |29.17 MB|

Classes 'data.table' and 'data.frame':	258125 obs. of  3 variables:
 $ nppes_provider_zip: Factor w/ 369898 levels "0000","00000",..: 157355 126839 92059 90279 111995 4059 64438 61626 70232 198240 ...
 $ frequency         : int  21295 16185 7075 7024 6926 6447 6384 6274 6029 5995 ...
 $ percent           : num  0.2326 0.1768 0.0773 0.0767 0.0757 ...
 - attr(*, ".internal.selfref")=<externalptr> 
NULL


|nppes_provider_zip|frequency|percent |
|:-----------------|:--------|:-------|
|559050001         |21295    |0.232649|
|441950001         |16185    |0.176822|
|326103003         |7075     |0.077295|
|322241865         |7024     |0.076738|
|372320001         |6926     |0.075667|
|018050001         |6447     |0.070434|


|nppes_provider_zip|frequency|percent |
|:-----------------|:--------|:-------|
|178151567         |1      

frequencies on a data.table with a column transformation to determine na's. Over 18M detail records had na's for average_medicare_standard_amt.

In [40]:
tic()

dt = "medicaref"
cols = "isna_average_medicare_standard_amt=is.na(average_medicare_standard_amt)"

fc <- freqsdt(dt,cols)

meta(fc,data=TRUE)

toc()

blanks(2)



|name|class                 |rows|columns|size   |
|:---|:---------------------|:---|:------|:------|
|fc  |data.table, data.frame|2   |3      |1.34 KB|

Classes 'data.table' and 'data.frame':	2 obs. of  3 variables:
 $ isna_average_medicare_standard_amt: logi  FALSE TRUE
 $ frequency                         : int  48338402 18441148
 $ percent                           : num  72.4 27.6
 - attr(*, ".internal.selfref")=<externalptr> 
NULL


|isna_average_medicare_standard_amt|frequency|percent |
|:---------------------------------|:--------|:-------|
|FALSE                             |48338402 |72.38504|
|TRUE                              |18441148 |27.61496|

0.55 sec elapsed

 


How do these na's break down by year?. No average_medicare_standard_amt values for 2012-2013.

In [41]:
tic()

dt = "medicaref"
cols = "year,isna_average_medicare_standard_amt=is.na(average_medicare_standard_amt)"

fc <- freqsdt(dt,cols)

meta(fc,data=TRUE)

toc()

blanks(2)



|name|class                 |rows|columns|size   |
|:---|:---------------------|:---|:------|:------|
|fc  |data.table, data.frame|7   |4      |1.57 KB|

Classes 'data.table' and 'data.frame':	7 obs. of  4 variables:
 $ year                              : int  2018 2017 2016 2015 2014 2013 2012
 $ isna_average_medicare_standard_amt: logi  FALSE FALSE FALSE FALSE FALSE TRUE ...
 $ frequency                         : int  9961865 9847443 9714896 9497891 9316307 9287876 9153272
 $ percent                           : num  14.9 14.7 14.5 14.2 14 ...
 - attr(*, ".internal.selfref")=<externalptr> 
NULL


|year|isna_average_medicare_standard_amt|frequency|percent |
|:---|:---------------------------------|:--------|:-------|
|2018|FALSE                             |9961865  |14.91754|
|2017|FALSE                             |9847443  |14.74620|
|2016|FALSE                             |9714896  |14.54771|
|2015|FALSE                             |9497891  |14.22275|
|2014|FALSE                

Tabulate frequencies for all medicaref factor and integer fields except npi, the key. The listing at the end provides cardinalities of each summarized attribute.

In [42]:
tic()

key = "npi"
nmes <- names(medicaref)
invars <- nmes[map_lgl(nmes,function(n) ifelse(class(medicaref[[n]]) %in% c("character","factor", "integer"),TRUE,FALSE))]
invars <- setdiff(invars,key) 

fcc <- data.table()
dt <- "medicaref"

for (n in invars)
{
    fc = freqsdt(dt,paste0(n))
    fcc = rbind(fcc,data.table(var=factor(names(fc)[1]),values=fc[[names(fc)[1]]],fc[,.(frequency,percent)]))

}

meta(fcc,data=TRUE)
blanks(2)
print(fcc[,.N,.(var)])

toc()
                       
blanks(2)



|name|class                 |rows   |columns|size     |
|:---|:---------------------|:------|:------|:--------|
|fcc |data.table, data.frame|1489998|4      |133.28 MB|

Classes 'data.table' and 'data.frame':	1489998 obs. of  4 variables:
 $ var      : Factor w/ 21 levels "nppes_provider_last_org_name",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ values   : Factor w/ 1438843 levels "10 33 AMBULANCE SERVICE LIMITED",..: 154372 189842 115677 96904 217603 134741 27065 221641 97104 184171 ...
 $ frequency: int  360942 341765 245430 243664 240628 210675 181909 178452 173221 166516 ...
 $ percent  : num  0.54 0.512 0.368 0.365 0.36 ...
 - attr(*, ".internal.selfref")=<externalptr> 
NULL


|var                         |values     |frequency|percent |
|:---------------------------|:----------|:--------|:-------|
|nppes_provider_last_org_name|PATEL      |360942   |0.540498|
|nppes_provider_last_org_name|SMITH      |341765   |0.511781|
|nppes_provider_last_org_name|LEE        |245430   |0.367523|
|nppes_prov

na's by year for each attribute.

In [43]:
tic()

dt <- "medicaref"

for (n in names(medicaref)) 
{
    print(freqsdt(dt,paste0("year,",paste0("isna_",n,'=is.na(',n,')'))))
    blanks(1)
}

toc()

blanks(2)

   year isna_npi frequency  percent
1: 2018    FALSE   9961865 14.91754
2: 2017    FALSE   9847443 14.74620
3: 2016    FALSE   9714896 14.54771
4: 2015    FALSE   9497891 14.22275
5: 2014    FALSE   9316307 13.95084
6: 2013    FALSE   9287876 13.90826
7: 2012    FALSE   9153272 13.70670

    year isna_nppes_provider_last_org_name frequency        percent
 1: 2018                             FALSE   9961800 14.91744104295
 2: 2017                             FALSE   9847382 14.74610415913
 3: 2016                             FALSE   9714834 14.54761824541
 4: 2015                             FALSE   9497816 14.22264151226
 5: 2014                             FALSE   9316237 13.95073342064
 6: 2013                             FALSE   9287797 13.90814553258
 7: 2012                             FALSE   9153195 13.70658382694
 8: 2013                              TRUE        79  0.00011829969
 9: 2012                              TRUE        77  0.00011530476
10: 2015                       

Finally, read the two parquet files back into data.tables. The factor version is much faster than the string. Parquet indeed seems a compact and efficient way to share data. I've successfully loaded these files into both pandas and Julia dataframes.

In [44]:
tic()

fn <- "medicarerf.parquet"
medicaref <- read_parquet(fn)

meta(medicaref)

toc()

blanks(2)



|name     |class                 |rows    |columns|size   |
|:--------|:---------------------|:-------|:------|:------|
|medicaref|data.table, data.frame|66779550|30     |9.55 GB|

Classes 'data.table' and 'data.frame':	66779550 obs. of  30 variables:
 $ npi                             : int  1003000126 1003000126 1003000126 1003000126 1003000126 1003000126 1003000126 1003000134 1003000134 1003000134 ...
 $ nppes_provider_last_org_name    : Factor w/ 318343 levels "10 33 AMBULANCE SERVICE LIMITED",..: 60738 60738 60738 60738 60738 60738 60738 37340 37340 37340 ...
 $ nppes_provider_first_name       : Factor w/ 81568 levels "","(BARBARA)",..: 3265 3265 3265 3265 3265 3265 3265 48840 48840 48840 ...
 $ nppes_provider_mi               : Factor w/ 34 levels "","'","(","-",..: 1 1 1 1 1 1 1 19 19 19 ...
 $ nppes_credentials               : Factor w/ 22866 levels "","(D.C.) CHIROPRACTIC",..: 5088 5088 5088 5088 5088 5088 5088 5088 5088 5088 ...
 $ nppes_provider_gender           : Factor w

In [45]:
tic()

fn <- "medicarers.parquet"
medicares <- read_parquet(fn)

meta(medicares)

toc()

blanks(2)



|name     |class                 |rows    |columns|size    |
|:--------|:---------------------|:-------|:------|:-------|
|medicares|data.table, data.frame|66779550|30     |14.02 GB|

Classes 'data.table' and 'data.frame':	66779550 obs. of  30 variables:
 $ npi                             : int  1003000126 1003000126 1003000126 1003000126 1003000126 1003000126 1003000126 1003000134 1003000134 1003000134 ...
 $ nppes_provider_last_org_name    : chr  "ENKESHAFI" "ENKESHAFI" "ENKESHAFI" "ENKESHAFI" ...
 $ nppes_provider_first_name       : chr  "ARDALAN" "ARDALAN" "ARDALAN" "ARDALAN" ...
 $ nppes_provider_mi               : chr  "" "" "" "" ...
 $ nppes_credentials               : chr  "M.D." "M.D." "M.D." "M.D." ...
 $ nppes_provider_gender           : chr  "M" "M" "M" "M" ...
 $ nppes_entity_code               : chr  "I" "I" "I" "I" ...
 $ nppes_provider_street1          : chr  "900 SETON DR" "900 SETON DR" "900 SETON DR" "900 SETON DR" ...
 $ nppes_provider_street2          : chr  "" 

That's all for now. Be back soon.